In [2]:
import os
os.chdir('../')

Elastic Net Regression suing Polynomial features and MICE Imputation.

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge

data = pd.read_csv("Datasets/analysis_data.csv")

y = data["monthly_spend"]
X = data.drop(columns=["monthly_spend"])

categorical_columns = X.select_dtypes(include=['object', 'category']).columns.tolist()
numeric_columns = X.select_dtypes(include=['number']).columns.tolist()

# OneHot Encoding
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_columns)
    ],
    remainder="passthrough"
)

X_encoded = preprocessor.fit_transform(X)
encoded_cat_cols = preprocessor.named_transformers_["cat"].get_feature_names_out(categorical_columns)
new_cols = list(encoded_cat_cols) + numeric_columns

X_encoded_df = pd.DataFrame(X_encoded, columns=new_cols)

# MICE Imputation using BayesianRidge
mice = IterativeImputer(
    estimator=BayesianRidge(),
    max_iter=10,
    initial_strategy='median',
    random_state=42
)

X_imputed = mice.fit_transform(X_encoded_df)
X_imputed_df = pd.DataFrame(X_imputed, columns=new_cols)


Getting polynomial features

In [4]:
from sklearn.preprocessing import PolynomialFeatures

# Polynomial features 
poly = PolynomialFeatures(
    degree=2,
    interaction_only=True,   
    include_bias=False
)

X_poly = poly.fit_transform(X_imputed_df)
poly_feature_names = poly.get_feature_names_out(X_imputed_df.columns)


Model fitting

In [6]:
from sklearn.linear_model import ElasticNetCV

enet = ElasticNetCV(
    l1_ratio=[0.1, 0.3, 0.5, 0.7, 0.9],
    alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10],
    cv=5,
    random_state=42,
    max_iter=5000
)

enet.fit(X_poly, y)

print("Best alpha:", enet.alpha_)
print("Best l1_ratio:", enet.l1_ratio_)


c:\Users\athar\anaconda4\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 999260385.0052273, tolerance: 927722.8243141122
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\athar\anaconda4\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 996537073.2317411, tolerance: 927722.8243141122
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\athar\anaconda4\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 993197345.0959721, tolerance: 927722.8243141122
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\athar\anaconda4\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:683: Co

Best alpha: 10.0
Best l1_ratio: 0.9


c:\Users\athar\anaconda4\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.250e+09, tolerance: 1.155e+06
  model = cd_fast.enet_coordinate_descent(


Testing on training data

In [7]:
from sklearn.metrics import root_mean_squared_error

y_hat=enet.predict(X_poly)
root_mean_squared_error(y,y_hat)

249.84511326424277

In [8]:
# After enet.fit(...)
coef = enet.coef_
features = poly.get_feature_names_out(new_cols)

significant_vars = pd.DataFrame({
    "feature": features,
    "coef": coef
})

significant_vars = significant_vars[significant_vars["coef"] != 0]
significant_vars = significant_vars.sort_values("coef", ascending=False)

print("\nTop significant variables:")
print(significant_vars)


Top significant variables:
                                       feature      coef
601                    tenure num_credit_cards  0.621494
602     num_transactions avg_transaction_value  0.461252
538                       age num_credit_cards  0.246173
583                  credit_score num_children  0.184484
483            card_type_platinum credit_score  0.167728
..                                         ...       ...
616      online_shopping_freq travel_frequency -0.093339
241   education_level_high school credit_score -0.093872
448  employment_status_unemployed credit_score -0.110525
154         marital_status_single credit_score -0.118483
499            card_type_standard credit_score -0.179356

[185 rows x 2 columns]


Scoring data

In [9]:
scoring = pd.read_csv("Datasets/scoring_data.csv")

# Apply same OneHot encoder
X_scoring_encoded = preprocessor.transform(scoring)
numeric_scoring_cols = scoring.select_dtypes(include=['number']).columns.tolist()
X_scoring_encoded_df = pd.DataFrame(X_scoring_encoded, columns=new_cols)

# MICE imputation
X_scoring_imputed = mice.transform(X_scoring_encoded_df)
X_scoring_imputed_df = pd.DataFrame(X_scoring_imputed, columns=new_cols)

# Polynomial features
X_scoring_poly = poly.transform(X_scoring_imputed_df)

# Predict
pred = enet.predict(X_scoring_poly)

# Export submission
submission = pd.DataFrame({
    "customer_id": scoring["customer_id"],
    "monthly_spend": pred
})
submission.to_csv("Submissions/submission_file_10.csv", index=False)
